In [33]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

def veri_cek():
    # Kullanıcıdan kaydırma sayısını iste
    sayfa = int(input("Scroll sayısını girin= "))
    browser = webdriver.Chrome()
    browser.get('https://www.google.com.tr/')
    time.sleep(1)
    wait = WebDriverWait(browser, 5)
    
    # Google'da arama işlemi
    yazi_girisi = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.gLFyf")))
    yazi_girisi.send_keys("X kullaniciadi")
    time.sleep(1)
    yazi_girisi.send_keys(Keys.ENTER)

    # İlk arama sonucuna tıkla
    tikla = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".yuRUbf")))
    tikla.click()

    time.sleep(2)

    # Giriş yap butonuna tıkla
    giris = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-testid='login']")))
    giris.click()
    
    # Kullanıcı adı gir (burada 'your_username' yerine kendi kullanıcı adınızı girin)
    kullanici_adi_alani = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='text']")))
    kullanici_adi_alani.send_keys("your_username")
    time.sleep(1)
    
    ileri_butonu = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='İleri']/ancestor::button")))
    ileri_butonu.click()
    time.sleep(1)

    # Telefon numarası girişinin gerekli olup olmadığını kontrol et
    try:
        telefon_veya_eposta_alani = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='text']")))
        telefon_veya_eposta_alani.send_keys("your_phone_number")  # gerekirse buraya telefon numaranızı girin
        time.sleep(1)

        # İleri butonuna tıklayın
        ileri_butonu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='ocfEnterTextNextButton']")))
        ileri_butonu.click()
        time.sleep(1)
        
        # Şifreyi gir (burada 'your_password' yerine kendi şifrenizi girin)
        sifre_alani = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='password']")))
        sifre_alani.send_keys("your_password")
        time.sleep(1)
        
        giris_yap_butonu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='LoginForm_Login_Button']")))
        giris_yap_butonu.click()

    except:
        # Telefon numarası gerekmiyorsa, doğrudan şifreyi gir
        sifre_alani = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name='password']")))
        sifre_alani.send_keys("your_password")  # kendi şifrenizle değiştirin
        time.sleep(1)
        
        giris_yap_butonu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='LoginForm_Login_Button']")))
        giris_yap_butonu.click()

    # Tweet verilerini bir CSV dosyasına yazma işlemi
    with open("tweetler.csv", "w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["tweetler", "yorum_sayisi", "retweet_sayisi", "begeni_sayisi"])

        a = 0
        scroll_percentage = 0.3
        
        while a < sayfa:
            lastHeight = browser.execute_script("return document.body.scrollHeight")
            
            # Sayfayı belirli bir yüzde oranında kaydır
            browser.execute_script(f"window.scrollBy(0, document.body.scrollHeight * {scroll_percentage});")
            time.sleep(1.5)  # Sayfanın yüklenmesini bekle
            newHeight = browser.execute_script("return document.body.scrollHeight")

            # Eğer sayfanın sonuna ulaşılmışsa, kaydırmayı durdur
            if newHeight == lastHeight:
                break
            else:
                lastHeight = newHeight
            
            # Sayfa kaynağını al ve BeautifulSoup ile ayrıştır
            sayfa_kaynagi = browser.page_source
            soup = BeautifulSoup(sayfa_kaynagi, "html.parser")
            
            # Tweetleri bul
            tweetler = soup.find_all("div", attrs={"data-testid": "cellInnerDiv"})
            
            for i in tweetler:
                # Tweet metni (boşluk kontrolü)
                tweet_text_element = i.find("div", attrs={"class": "css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-1qd0xha r-a023e6 r-rjixqe r-16dba41 r-bnwqim"})
                tweet_text = tweet_text_element.text if tweet_text_element else "TWEET METNİ BULUNMUYOR"

                # Ek bilgiler (boşluk kontrolü)
                ek_bilgi_element = i.find("div", attrs={"class": "css-175oi2r r-1d09ksm r-18u37iz r-1wbh5a2"})
                ek_bilgi = ek_bilgi_element.text if ek_bilgi_element else ""

                # Yorum sayısı (0 kontrolü)
                yorum_sayisi_element = i.find("button", attrs={"data-testid": "reply"})
                yorum_sayisi = yorum_sayisi_element.text if yorum_sayisi_element and yorum_sayisi_element.text else "0"

                # Retweet sayısı (0 kontrolü)
                retweet_sayisi_element = i.find("button", attrs={"data-testid": "retweet"})
                retweet_sayisi = retweet_sayisi_element.text if retweet_sayisi_element and retweet_sayisi_element.text else "0"

                # Beğeni sayısı (0 kontrolü)
                begeni_sayisi_element = i.find("button", attrs={"data-testid": "like"})
                begeni_sayisi = begeni_sayisi_element.text if begeni_sayisi_element and begeni_sayisi_element.text else "0"

                # Tweet verisini CSV'ye yaz
                writer.writerow([tweet_text, yorum_sayisi, retweet_sayisi, begeni_sayisi])
            
            a += 1  

veri_cek()


Scroll sayısını girin=  500
